# 佐賀県の人口推移

In [ ]:
import pandas
import requests
import io
import matplotlib.pyplot as plt

In [ ]:
#エクセル中の和暦ラベルと西暦の対応付け
wareki={'大正9年':1920, '昭和5年':1930, '10年':1935, '15年':1940,
        '20年':1945, '25年':1950, '30年':1955, '35年':1960,'40年':1965,
        '45年':1970, '50年':1975, '55年':1980, '60年':1985, '平成2年':1990,
        '7年':1995, '12年':2000, '17年':2005, '22年':2010,'27年':2014}

df中の指定された行のSeriesをプロット

In [ ]:
def plotSub(pList,df):
    for loc,label in pList:
        ser=evalPop(df,loc)
        ser.plot(label=label)

人口推移をSeriesとして返す

- 人口を千人単位に変更
- 和暦を西暦に変更

In [ ]:
def evalPop(data,serName):
    vList,yList=[],[]
    for k in wareki.keys():
        p = data[k][serName]/1000.
        y = wareki[k]#和暦から西暦へ
        vList.append(p)
        yList.append(y)
    return pandas.Series(vList,index = yList)

作図前処理

In [ ]:
def prePlot():
    plt.figure(figsize=(10,10))
    plt.title('Population in Saga (from National Census)')#図タイトル
    plt.xlabel('year')
    plt.ylabel('population (thousand)')

作図後処理：作図情報などを表示

In [ ]:
def postPlot(labelList=[]):#引数を省略すると空リスト
    #図の書式など
    plt.legend(loc='best')#凡例
    for x,y,label in labelList:
        plt.text(x,y,label,horizontalalignment='center')

メイン

In [ ]:
url = 'http://data.bodik.jp/dataset/'\
       '77e0cc66-c15d-4473-b3df-2664fe8e2e63/resource/'\
       '8dc71515-526a-4168-866c-05d2cc8dad7b/download/jinkou.xlsx'

data = pandas.read_excel(url,header=3)#Excelの内容をDataFrameへ

prePlot():

pList=[(2,'Total'),(3,'Cities'),(4,'Counties')]
plotSub(pList,data)
labelList=[(1953,400,'1953'),(2005,600,'2005')]

postPlot(labelList)

plt.savefig('output.png')#画像保存
plt.show()#画像表示